In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="../../data/",
    train=True,
    download=True,
    transform=ToTensor()
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="../../data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: {}".format(X.shape))
    print("dtype of y: {}".format(y.dtype))
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
dtype of y: torch.int64


In [5]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print("loss: {:.7f} [{:5d}/{:5d}]".format(loss, current, size))

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_loss += loss_fn(pred, y).item()
    print("Test Error: \n Accuracy: {:.0.1f}%, Avg loss: {:.8f} \n")

epochs = 5
for t in range(epochs):
  print("Epoch {}\n---------------------------".format(t+1))
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss: 2.3119755 [   64/60000]
loss: 2.2856956 [ 6464/60000]
loss: 2.2726328 [12864/60000]
loss: 2.2629507 [19264/60000]
loss: 2.2415323 [25664/60000]
loss: 2.2241042 [32064/60000]
loss: 2.2299955 [38464/60000]
loss: 2.1991470 [44864/60000]
loss: 2.2006071 [51264/60000]
loss: 2.1671858 [57664/60000]
pred is  tensor([[-6.7455e-02, -1.2911e-01, -6.1509e-02, -1.4454e-01, -3.0750e-02,
         -6.6012e-02, -9.3463e-02,  8.7123e-03,  1.3653e-01,  1.6672e-01],
        [ 1.3143e-01, -2.4612e-01,  3.0631e-01, -9.6586e-02,  3.5907e-01,
         -3.1469e-01,  2.4118e-01, -4.4607e-01,  1.6019e-01,  1.5148e-02],
        [ 1.3900e-01,  2.5242e-01,  1.3497e-01,  9.6837e-02, -2.8431e-03,
         -2.8046e-01,  9.6844e-02, -2.4942e-01, -3.6534e-02, -1.1884e-01],
        [ 9.5328e-02,  1.6042e-01,  9.5732e-02,  6.5954e-02,  1.5380e-02,
         -2.0977e-01,  7.8199e-02, -1.9974e-01, -2.4628e-02, -8.2192e-02],
        [ 1.2877e-01, -1.2430e-01,  1.5744e-01, -5.1836e-02

In [10]:
# Save Models
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# Loading Models
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print('Predicted: "{}", Actual: "{}"'.format(predicted, actual))

Saved PyTorch Model State to model.pth
Predicted: "Ankle boot", Actual: "Ankle boot"
